In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
from statsmodels.tsa.arima.model import ARIMA

In [ ]:
def smape(predictions, actual):
    difference = np.abs(predictions-actual)
    summation = np.abs(actual)+np.abs(predictions)
    error = np.mean(difference/summation)
    return error

First we will do some simple tuning of ARIMA model.

In [ ]:
list_of_names = ["linear1_abrupt", "linear2_abrupt", "linear3_abrupt",
                "nonlinear1_abrupt", "nonlinear2_abrupt", "nonlinear3_abrupt"]

In [ ]:
for iteration in range(0,6):
    smape_dict = {}
    for name in list_of_names:
        data = pd.read_csv("data/"+name, usecols = [iteration]).iloc[:,0].to_list()

        # split data into train and test
        split = int(len(data) * 0.7)
        train, test = data[:split], data[split:]
        history = [x for x in train]

        # make predictions
        predictions = []
        start = time.perf_counter()
        for t in range(len(test)):
            model = ARIMA(history, order=(5,1,0))
            model_fit = model.fit()
            yhat = model_fit.forecast()[0]
            predictions.append(yhat)
            history.append(test[t])
        # calculate out of sample error
        error = smape(np.asarray(predictions), np.asarray(test))
        smape_dict[name] = error
        end = time.perf_counter()
#         show plots only first time
#         plt.plot(test, label = "expected", color = "black")
#         plt.plot(predictions, label = "predicted", color = "red")
#         plt.title(name)
#         plt.legend()    

#         #saving the plots
#         image_path = "forecasting_results/arima/"+name+".png"
#         plt.savefig(image_path)

#         plt.show()

    #saving the dictionary containing errors
    dict_path = "forecasting_results/arima/errors/error"+str(iteration)+".txt"
    with open(dict_path, 'w') as file:
        for key in smape_dict.keys():
            file.write("%s,%s\n"%(key,smape_dict[key]))